In [1]:
import csv
import datetime
import requests
from bs4 import BeautifulSoup as bs4
import re

search_word="伊丹"

csv_date = datetime.datetime.today().strftime("%Y%m%d")
csv_filename = "goteat_itami" + csv_date + ".csv"
f = open(csv_filename, mode = 'w', encoding='utf-8', errors = 'ignore')
writer = csv.writer(f, lineterminator = '\n')

# CSVヘッダー
csv_header = ["店名 ", "住所", "電話", "営業時間", "Webサイト", "メモ"]
writer.writerow(csv_header)

for i in range(100):
    page = i + 1
    # html情報取得・解析
    url ="https://gotoeat-hyogo.com/search/result"
    param = {"keyword" : search_word, "page" : page} #「伊丹」検索
    res = requests.get(url, param)
    search_result = bs4( res.text, "html.parser")
    
    # 店舗名、店舗住所の取得
    store_list = search_result.select("div.search-results-list-box")
    if(len(store_list) == 0):
        print("page:", i)
        break
    
    for store in store_list:
        #店名
        store_name = store.select("p.search-results-list-name")[0].text
        #住所
        store_address = store.select("p.search-results-list-p01")[0].text.replace(" ", "").replace("\n", "").replace("住所：", "")
        store_address = re.sub('(〒[0-9]{3}-[0-9]{4})', "", store_address)
        #電話番号
        store_tel_number = store.select("p.search-results-list-p01")[1].text.replace(" ", "").replace("\n", "").replace("TEL:", "")
        
        #詳細ページ解析
        store_detail_url = store.select("p.search-results-list-btn a")[0]["href"]
        store_res = requests.get(store_detail_url)
        store_detail = bs4( store_res.text, "html.parser")
        store_detail_table = store_detail.select("table.shop-detail-info-table")
        
        #営業時間
        store_business_hours_title = store_detail_table[0].find("th", text="営業時間")
        store_business_hours = store_business_hours_title.next_sibling.next_sibling.text if store_business_hours_title else ""
        #Webサイト
        store_web_title = store_detail_table[0].find("th", text="URL")
        store_web = store_web_title.next_sibling.next_sibling.text if store_web_title else ""
        #メモ(定休日)
        store_regular_holiday_title = store_detail_table[0].find("th", text="定休日")
        store_memo = "定休日:" + store_regular_holiday_title.next_sibling.next_sibling.text if store_regular_holiday_title else ""
        
        csv_list = []
        csv_list.append(store_name)
        csv_list.append(store_address)
        csv_list.append(store_tel_number)
        csv_list.append(store_business_hours)
        csv_list.append(store_web)
        csv_list.append(store_memo)
        writer.writerow(csv_list)

f.close()

page: 10
